# `ASFProduct` Subclasses

`ASFProduct` is the base class for all search result objects as of asf-search v7.0.0. There are several subclasses of `ASFProduct` that asf-search uses for specific platforms and product types with unique properties/functionality.

Key Methods:
- `geojson()`
- `download()`
- `stack()`
- `get_stack_opts()` (returns None in `ASFProduct`, implemented by `ASFStackableProduct` subclass and its subclasses)
- `centroid()`
- `remotezip()` (requires asf-search's optional dependency be installed)
- `translate_product()` (reads properties from umm, populates `properties` with associated keyword)
- `get_sort_keys()`
- `umm_get()`

Key Properties:
- `properties`
- `_base_properties` (maps `properties` keys to values in umm json)
- `umm` (The product's umm JSON from CMR)
- `metadata` (The product's metadata JSON from CMR)

In [ ]:
import asf_search as asf
products = ['S1A_IW_SLC__1SDV_20231226T162948_20231226T163016_051828_0642C6_272F-SLC', 'S1_185682_IW2_20210224T161634_VV_035E-BURST','S1-GUNW-D-R-087-tops-20190301_20190223-161540-20645N_18637N-PP-7a85-v2_0_1-unwrappedPhase','ALPSRP111041130-RTC_HI_RES', 'UA_newyor_03204_22005-013_22010-002_0014d_s01_L090_01-INTERFEROMETRY']
results = asf.product_search(product_list=products)
results

Notice the different type in the `results` list: `S1Product`, `S1BurstProduct`, `ARIAS1GUNWProduct`, `ALOSProduct`, and `UAVSARProduct`.
Each of these classes are subclassed from `ASFProduct` in some way.

Let's compare the `properties` of `S1Product` and `ALOSProduct`

In [ ]:
s1, uavsar, s1Burst, ariaGunw, alos = results

def compare_properties(lhs: asf.ASFProduct, rhs: asf.ASFProduct):
    # Compares properties of two ASFProduct objects in a color coded table
    # values printed in red are missing from that product type altogether
    
    # Color Coding
    RED = '\033[31m'
    GREEN = '\033[32m'
    BLUE  = '\033[34m'
    RESET = '\033[0m'

    print(f'\t{GREEN}{type(lhs)}{RESET}\t{BLUE}{type(rhs)}{RESET}')
    
    keys = {*lhs.properties.keys(), *rhs.properties.keys()}
    for key in keys:
        print(f"{key}:\n\t{GREEN}{lhs.properties.get(key, f'{RED}None')}{RESET}\t{BLUE}{rhs.properties.get(key, f'{RED}None')}{RESET}\n")

compare_properties(s1, uavsar)

Notice a few properties (marked in red) are missing from each product properties dict. For example, `S1Product` has `pgeVersion`, while `UAVSARProduct` has `insarStackId`. 

Moreover, `S1Product` has one major difference with `UAVSARProduct`: `S1Product` inherits from `ASFStackableProduct` (see section below).

In [ ]:
print(f"{s1.properties['fileID']}\n\t{s1.baseline}\n")
print(f"{uavsar.properties['fileID']}\n\t{uavsar.baseline}")

# `ASFStackableProduct`

`ASFStackableProduct` is an important `ASFProduct` subclass, from which stackable products types meant for time-series analysis are derived from. `ASFStackableProduct` has a class enum, `BaselineCalcType` that determines how asf-search will handle perpendicular stack calculations. Each subclass keeps track of their baseline calculation type via the `baseline_type` property.

Inherits: `ASFProduct`

Inherited By:
- `ALOSProduct`
- `ERSProduct`
- `JERSProduct`
- `RADARSATProduct`
- `S1Product`
    - `S1BurstProduct`
    - `OPERAS1Product` (Stacking currently disabled)
    - `ARIAS1GUNWProduct` (Stacking currently disabled)

Key Methods:
- `get_baseline_calc_properties()`
- `get_stack_opts()` (Overrides `ASFproduct`)
- `is_valid_reference()`
- `get_default_baseline_product_type()`

Key Definitions:
class enum `BaselineCalcType`:
- `PRE_CALCULATED` Has pre-calculated `insarBaseline` value that will be used for perpendicular calculations
- `CALCULATED` Uses position/velocity state vectors and ascending node time for perpendicular calculations

Key Fields:
- `baseline`
- `baseline_type` (`BaselineCalcType.PRE_CALCULATED` by default or `BaselineCalcType.CALCULATED`)



In [ ]:
print(f"Baseline Calculation Types")
print(f"ASFProduct:\t {asf.ASFStackableProduct.baseline_type}")
print(f"ALOSProduct:\t {alos.baseline_type}")
print(f"S1Product:\t {s1.baseline_type}")

`ASFStackableProduct` subclasses even have their own stack search option methods. The `ASFStackableProduct` implementation of `get_stack_opts()` returns the commonly used params for pre-calculated datasets (processing level and insar stack ID), but subclasses like `S1Product` and `S1BurstProduct` use their own approach. 

In [ ]:
print(f"S1Product:\n{s1.get_stack_opts()}\n")
print(f"S1BurstProduct:\n{s1Burst.get_stack_opts()}\n")
print(f"ALOSProduct:\n{alos.get_stack_opts()}")

# Writing Custom `ASFProduct` Subclasses
Because `ASFProduct` is built for subclassing, that means users can provide their own custom subclasses dervied directly from `ASFProduct` or even from a pre-existing subclass like `S1Product` or `OperaS1Product`.

In this example we subclass `S1Product`, and overrides the default `ASFProduct.stack()` with one that returns a _list_ of `S1BurstProduct` stacks based on an area of interest, modify `geojson()` to return state vectors, and add a new helper method for getting raw umm CMR response!

In [ ]:
import copy
from typing import List, Type, Union, Dict
from asf_search import ASFSearchOptions, ASFSession, ASFSearchResults
from asf_search.ASFSearchOptions import ASFSearchOptions
from asf_search.CMR.translate import try_parse_int
from datetime import datetime

class MyCustomS1Subclass(asf.S1Product):
    def __init__(
                #default ASFProduct constructor arguments
                self, args: dict = {}, session: ASFSession = ASFSession()
            ):
        super().__init__(args, session)

        # totaly unique property of MyCustomClass
        self.timestamp = datetime.now()

    # _base_properties is a special dict of ASFProduct that maps keywords to granule UMM json
    # defining properties and their paths here will let you
    # easily access them in the product's `properties` dictionary
    # see `ASFProduct.umm_get()` for explanation of pathing
    _base_properties = {
        # Most product types use `CENTER_ESA_FRAME` as the value for `frameNumber` (unlike S1 and ALOS, which use `FRAME_NUMBER`), 
        # this creates a new `esaFrame` property so we have that value too
        **asf.S1Product._base_properties,
        'esaFrame': {'path': ['AdditionalAttributes', ('Name', 'CENTER_ESA_FRAME'), 'Values', 0], 'cast': try_parse_int}, #Sentinel and ALOS product alt for frameNumber (ESA_FRAME)
    }

    """ Example umm that the above pathing would map to:
        'umm': {
            'AdditionalAttributes': [
                {
                    'Name': 'CENTER_ESA_FRAME',
                    "Values": ['1300'] 
                },
                ...
            ],
            ...
        }
    """

    # CUSTOM CLASS METHODS
    # Return
    def as_umm_json(self) -> Dict:
        return { 'umm': self.umm, 'meta': self.meta }
    
    # CLASS OVERRIDE METHODS
    
    # This override of `geojson()` includes the product's state vectors in the final geojson output, 
    # along with a custom class field timestamp and what version of asf-search was used at runtime
    def geojson(self) -> dict:
        output = super().geojson()

        output['properties']['stateVectors'] = self.get_state_vectors()
        output['properties']['timestamp'] = str(self.timestamp)
        output['properties']['ASFSearchVersion'] = asf.__version__
        return output

    # ASFProduct.stack() normally stacks the current product
    # in this version we search for every SLC-BURST product that
    # overlaps the given area with the same source scene, 
    # and return a list of burst stacks
    # if no bursts are found, we fall back to building a regular stack
    def stack(self, 
                opts: ASFSearchOptions = None,
                useSubclass: Type[asf.ASFProduct] = None,
                aoi: str = None
            ) -> Union[ASFSearchResults, List[ASFSearchResults]]:

        bursts = asf.search(
            groupID=self.properties['groupID'], 
            processingLevel=asf.PRODUCT_TYPE.BURST,
            intersectsWith=aoi if aoi is not None else opts.intersectsWith
        )
        
        return [burst.stack(opts=opts) for burst in bursts]

In [ ]:

customS1SubclassProduct = MyCustomS1Subclass({'umm': s1.umm, 'meta': s1.meta}, session=s1.session)

customS1SubclassProduct.geojson()

Notice the `timestamp`, `ASFSearchVersion`, `stateVectors`, and `esaFrame` fields in the output from `geojson()`.
Below is a comparison of properties between the built-in `S1Product` and our `customS1SubclassProduct`.

In [ ]:
compare_properties(s1, customS1SubclassProduct)

In [ ]:
palmer_to_anchorage = 'LINESTRING(-149.1052 61.6054,-149.5376 61.3162,-149.8764 61.2122)'
customStack = customS1SubclassProduct.stack(aoi=palmer_to_anchorage)
customStack

Notice instead of a stack of `MyCustomS1Subclass` products we have a list of `S1BurstProduct` stacks!
Below is a breakdown of this list of stacks:

In [ ]:
from typing import List

def view_stack_of_stacks(stack_of_stacks: List):
    print(f'Found {len(stack_of_stacks)} SLC-BURST stacks over AOI, stack lengths:')
    for stack_idx, stack in enumerate(stack_of_stacks):
        print(f"\t{stack_idx+1}:\t{len(stack)} SLC-BURSTs \t(Full Burst ID: {stack[-1].properties['burst']['fullBurstID']}, polarization: {stack[-1].properties['polarization']})")

view_stack_of_stacks(customStack)

# Using Custom `ASFProduct` Subclasses in Baseline Search

There may be instances where you want to build a spatio-temporal baseline stack from a reference of a custom subclass. `stack_from_id()` and `ASFProduct.stack()` support this via the `ASFProductSubclass` keyword.

In [ ]:
opts = asf.ASFSearchOptions(intersectsWith=palmer_to_anchorage) # our custom class will be able to use our aoi this way

customSubclassStack = asf.stack_from_id('S1A_IW_SLC__1SDV_20231226T162948_20231226T163016_051828_0642C6_272F-SLC', opts=opts, useSubclass=MyCustomS1Subclass)

view_stack_of_stacks(customSubclassStack)